# Importación de librerias

In [3]:
import pandas as pd
from pandas import json_normalize


# Lectura de Json

In [4]:
customer_file = 'customers.json'
df_json_nested = pd.read_json(customer_file, lines= True)

In [5]:
df_json_nested.head(3)


,fraudulent,customer,orders,paymentMethods,transactions
0,False,"{'customerEmail': 'josephhoward@yahoo.com', 'c...","[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb..."
1,True,"{'customerEmail': 'evansjeffery@yahoo.com', 'c...","[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg..."
2,False,"{'customerEmail': 'andersonwilliam@yahoo.com',...","[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3..."


# Desanidar el json 

### Customers dataframe

In [6]:
#normalizar la columna 'customer' que es un json anidado
customers_df = json_normalize(df_json_nested['customer'])

### Orders Dataframe

In [7]:
#lista de ordenes anidadas
orders_nested = pd.DataFrame([md for md in df_json_nested["orders"]])

orders_list=[]
for index,row in orders_nested.iterrows():
    for order in row:
        if order != None:
            orders_list.append(order)           

orders_df = pd.DataFrame(orders_list)

### Transactions Dataframe

In [8]:
#lista de transactions anidadas
transactions_nested = pd.DataFrame([md for md in df_json_nested["transactions"]])

transactions_list=[]
for index,row in transactions_nested.iterrows():
    for transaction in row:
        if transaction != None:
            transactions_list.append(transaction)           
transactions_df = pd.DataFrame(transactions_list)

### Payment Dataframe

In [9]:
#Creating a payment_methods dataframe:
payment_nested = pd.DataFrame([md for md in df_json_nested["paymentMethods"]])

payment_methods_list=[]
for index,row in payment_nested.iterrows():
    for order in row:
        if order != None:
            payment_methods_list.append(order)           
payment_methods_df = pd.DataFrame(payment_methods_list)

### Fraudulent Dataframe

In [10]:
fraudulent_df = df_json_nested['fraudulent']

### concatenar los DF

In [11]:
df_flat = pd.concat([customers_df, orders_df, payment_methods_df, transactions_df, fraudulent_df], axis=1)

### borrar columnas duplicadas

In [12]:
df_flat = df_flat.loc[:,~df_flat.columns.duplicated()].copy()

### Columnas CustomerBillingAddress y OrderShipping address

In [13]:
def extract_city_state_name(name_column):
    city_list = []
    state_list = []

    for address in df_flat[name_column]:
        try:
            # Tomar la línea con ciudad y estado
            aux = address.split('\n')[1]  # Ej: 'Visalia, CA 51896'

            # Separar ciudad y estado+zip
            city = aux.split(',')[0].strip()  # Ej: 'Visalia'
            state = aux.split(',')[1].strip().split(' ')[0]  # Ej: 'CA'

            city_list.append(city)
            state_list.append(state)

        except:
            city_list.append("unknown")
            state_list.append("unknown")

    return city_list, state_list


In [14]:
#extraer el estado de la columna customerbillingaddress
df_flat["customerCitybillingaddress"], df_flat["customerStatebillingaddress"] = extract_city_state_name("customerBillingAddress")


In [15]:
#extraer el estado de la columna orderShippingAddress
df_flat["ordershippingaddressCity"], df_flat["ordershippingaddressState"] = extract_city_state_name("orderShippingAddress")


In [16]:
#comparar si las direcciones de facturacion y envio son iguales
same_city = []
same_state = []

for index in range(len(df_flat)):
    city_billing = df_flat["customerCitybillingaddress"][index]
    city_shipping = df_flat["ordershippingaddressCity"][index]
    state_billing = df_flat["customerStatebillingaddress"][index]
    state_shipping = df_flat["ordershippingaddressState"][index]

    # Comparación de ciudad
    if city_billing != "unknown" and city_shipping != "unknown":
        same_city.append("yes" if city_billing == city_shipping else "no")
    else:
        same_city.append("unknown")

    # Comparación de estado
    if state_billing != "unknown" and state_shipping != "unknown":
        same_state.append("yes" if state_billing == state_shipping else "no")
    else:
        same_state.append("unknown")


In [17]:
# Agregar la columna 'sameCity' al DataFrame df_flat
df_flat["samecity"] = same_city
df_flat["samestate"] = same_state


In [18]:
# Borrar las columnas para quedaros con samecity,same state
df_flat = df_flat.drop(columns=["customerCitybillingaddress","customerStatebillingaddress","ordershippingaddressCity","ordershippingaddressState","customerBillingAddress","orderShippingAddress"])


### Columna CustomerEmail

In [19]:
#obtener los dominios de los emails y categorizar los proveedores populares
email_domains= []
mails= []
popular_providers = ['yahoo', 'gmail', 'hotmail']

for email in df_flat['customerEmail']:
    try:
        aux = email.split('@')[1]
        domain = aux.split('.')[1]
        mail = aux.split('.')[0]
        
        if mail in popular_providers:
            mails.append(mail)
        else:
            mails.append('other')
        email_domains.append(domain)
        
    except:
        email_domains.append('weird')
        mails.append('weird')
        

In [20]:
#Crear columna emaildomain
df_flat['emailDomain'] = email_domains


In [21]:
#Crear columna emailprovider
df_flat['emailProvider'] = mails

In [22]:
df_flat = df_flat.drop(['customerEmail'], axis=1)

### Columna customerIPAddress

In [23]:
#manejo de direcciones IPv4 e IPv6
IP_addresses = []
for address in df_flat['customerIPAddress']:

    aux_address = str(address)
    
    if len(aux_address) > 15:
        IP_addresses.append('digits_and_letters')
    else:
        IP_addresses.append('only_letters')



In [24]:
#instanciamos las IP
df_flat['customerIPAddressSimplified'] = IP_addresses

In [25]:
df_flat = df_flat.drop(['customerIPAddress'], axis=1)

### Columna customerPhone

In [26]:
#borrar columna
df_flat = df_flat.drop(['customerPhone'], axis=1)

### Columnas con identificadores

In [27]:
df_flat = df_flat.drop(['customerDevice', 'orderId', 'transactionId', 'paymentMethodId'], axis=1)

In [28]:
df_flat.head(3)

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,samecity,samestate,emailDomain,emailProvider,customerIPAddressSimplified
0,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,18,False,False,yes,yes,com,yahoo,only_letters
1,26.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,26,False,True,no,no,com,yahoo,only_letters
2,45.0,fulfilled,False,card,VISA 16 digit,Vertex Bancorp,45,False,False,no,no,com,yahoo,digits_and_letters


# Guardar Dataset

In [29]:
filename = "customer_dataset.csv"
df_flat.to_csv(filename, index = False)